## 텍스트 유사도를 측정하는 fuzzzywuzzy 라이브러리로 두 가지 재밌는 과제를 해 보자

1. 뉴스 제목 리스트에서 유사 제목의 뉴스 걸러내기(중복제거)
2. YES24 서평 코멘트 리스트에서 표절 서평 찾아내기

* fuzzywuzzy 라이브러러 : 레빈스타인거리 알고리즘을 바탕으로 문장 유사도 계산

In [0]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process

import pandas as pd
import numpy as np
import copy

### 1. 시작에 앞서 예전 깃헙에 올린 샘플 살짝 복습하며 몸풀기  
https://github.com/YoonHanho/fuzzywuzzy_example/blob/master/fuzzywuzzy_example.ipynb

In [0]:
# 네이버 경제면 가장 많이 본 뉴스에서
# 키워드별로 유사한 뉴스를 찾아보자

news_titles = [
'한국서 일감 나눠먹기하다 걸린 일본차 부품사들···檢 고발',
'41억원 규모 재고 반품, 559명 인건비 떠넘긴 CJ올리브영',
'日에 칼 빼든 공정위, 자동차부품 담합한 4개 업체 적발',
'납품업체에 갑질 종합세트 CJ올리브영…과징금 10억 원',
'올리브영, CJ올리브네트웍스 납품사에 온갖 갑질…과징금 10억',
'남자가 마흔 넘으면 결혼하기 힘든 이유',
'일본 불매운동, 딜레마…유니클로 사지마 vs 일식집은 살려야?',
'"세븐일레븐은 한국 편의점"…일본 불매운동에 긴급 공지 발송한 세븐일레븐',
'[집코노미TV] "스타강사 말 믿고 버스타고 몰려다니다 망합니다"',
'日 자동차 부품사, 10년간 우리나라서 ‘납품담합’…과징']

keyword = "일본 불매 아배 반일"
process.extractOne(keyword, news_titles)

('일본 불매운동, 딜레마…유니클로 사지마 vs 일식집은 살려야?', 86)

In [0]:
# 뉴스별 유사도 값 산출
for news_title in news_titles:
  similar = fuzz.token_set_ratio(keyword, news_title)
  print(news_title, " ==> ", similar)

한국서 일감 나눠먹기하다 걸린 일본차 부품사들···檢 고발  ==>  20
41억원 규모 재고 반품, 559명 인건비 떠넘긴 CJ올리브영  ==>  14
日에 칼 빼든 공정위, 자동차부품 담합한 4개 업체 적발  ==>  15
납품업체에 갑질 종합세트 CJ올리브영…과징금 10억 원  ==>  15
올리브영, CJ올리브네트웍스 납품사에 온갖 갑질…과징금 10억  ==>  14
남자가 마흔 넘으면 결혼하기 힘든 이유  ==>  19
일본 불매운동, 딜레마…유니클로 사지마 vs 일식집은 살려야?  ==>  33
"세븐일레븐은 한국 편의점"…일본 불매운동에 긴급 공지 발송한 세븐일레븐  ==>  31
[집코노미TV] "스타강사 말 믿고 버스타고 몰려다니다 망합니다"  ==>  14
日 자동차 부품사, 10년간 우리나라서 ‘납품담합’…과징  ==>  15


### 2. 뉴스 제목 리스트에서 유사 제목의 뉴스 걸러내기(중복제거)

동일한 뉴스인데도, 언론사별로 조금씩 제목이 바뀌어 올라오는 경우 다반사   
fuzzywuzzy 라이브러리로 제목이 유사한 뉴스를 걸러내자

In [0]:
### 미래에셋대우 뉴스 헤드라인### 
news_titles = \
["[머니+ 베스트 컬렉션] 미래에셋대우 'MTS 해외송금 서비스'",
"김연추 미래에셋대우 에쿼티본부장 파생결합증권 획일화된 상품 구조 문제...",
"고광범 미래에셋대우 글로벌 이사 현지화 역량 강화해 직접 딜소싱 늘려...",
"미래에셋대우, 다이렉트 전용 ELS 100억원 규모로 판매",
"[머니팁]미래에셋대우, 100억 규모 ELS 1종 공모",
"미래에셋대우, 다이렉트 전용 ELS 판매",
"미래에셋대우, 다이렉트 전용 ELS 판매",
"미래에셋대우, 업계 최초 모바일 해외송금 서비스",
"미래에셋대우, 증권업계 처음으로 해외송금 서비스 개시",
"미래에셋대우, 증권업계 첫 MTS 해외송금서비스 시작",
"미래에셋대우, 증권업계 최초 MTS 해외송금 서비스 출시",
"미래에셋대우, 증권업계 최초 MTS 해외송금서비스 개시",
"미래에셋대우, 증권업계 최초 MTS 해외송금서비스",
"미래에셋대우, 증권업계 최초로 MTS 해외송금서비스 개시",
"미래에셋대우, 증권업계 최초 MTS 해외송금서비스 오픈",
"미래에셋대우, 증권업계 최초 MTS 해외송금서비스 오픈",
"미래에셋대우, 업계 첫 MTS 해외송금 서비스 개시",
"미래에셋대우, 투자자 제안 받아 펀드 출시 및 모바일 앱 개선",
"미래에셋대우, 온라인 고객 제안 접수...개선과제 도출",
"美 바이오 네오이뮨텍, IPO 추가 주관사 미래에셋대우 선정",
"미래에셋대우, 다이렉트 신용·담보대출 연 3.9% 이벤트",
"미래에셋대우, `다이렉트 신용융자·담보·매도담보대출 연 3.9% 이벤...",
"미래에셋대우, '다이렉트 신용융자·담보·매도담보대출 연3.9%' 이벤...",
"미래에셋대우, 투자성과 가시화 단계 진입-대신",
"[클릭 e종목]미래에셋대우, 해외IB·네이버·레버리지비율 '재료'충...",
"미래에셋대우, 22일 영통WM·광화문WM 투자설명회 개최",
"미래에셋대우 영통·광화문WM 투자설명회 개최",
"미래에셋대우, 영통·광화문WM 투자설명회 개최",
"미래에셋대우 영통·광화문WM, 22일 투자설명회 개최"]

유사도 값 분포 확인

In [0]:
keyword = news_titles[-8]
print("타겟문장 : {}\n".format(keyword))

for news_title in news_titles:
  similar = fuzz.token_set_ratio(keyword, news_title)
  print(news_title, " ==> ", similar)

타겟문장 : 미래에셋대우, `다이렉트 신용융자·담보·매도담보대출 연 3.9% 이벤...

[머니+ 베스트 컬렉션] 미래에셋대우 'MTS 해외송금 서비스'  ==>  41
김연추 미래에셋대우 에쿼티본부장 파생결합증권 획일화된 상품 구조 문제...  ==>  34
고광범 미래에셋대우 글로벌 이사 현지화 역량 강화해 직접 딜소싱 늘려...  ==>  37
미래에셋대우, 다이렉트 전용 ELS 100억원 규모로 판매  ==>  52
[머니팁]미래에셋대우, 100억 규모 ELS 1종 공모  ==>  39
미래에셋대우, 다이렉트 전용 ELS 판매  ==>  69
미래에셋대우, 다이렉트 전용 ELS 판매  ==>  69
미래에셋대우, 업계 최초 모바일 해외송금 서비스  ==>  39
미래에셋대우, 증권업계 처음으로 해외송금 서비스 개시  ==>  36
미래에셋대우, 증권업계 첫 MTS 해외송금서비스 시작  ==>  36
미래에셋대우, 증권업계 최초 MTS 해외송금 서비스 출시  ==>  38
미래에셋대우, 증권업계 최초 MTS 해외송금서비스 개시  ==>  35
미래에셋대우, 증권업계 최초 MTS 해외송금서비스  ==>  38
미래에셋대우, 증권업계 최초로 MTS 해외송금서비스 개시  ==>  35
미래에셋대우, 증권업계 최초 MTS 해외송금서비스 오픈  ==>  35
미래에셋대우, 증권업계 최초 MTS 해외송금서비스 오픈  ==>  35
미래에셋대우, 업계 첫 MTS 해외송금 서비스 개시  ==>  40
미래에셋대우, 투자자 제안 받아 펀드 출시 및 모바일 앱 개선  ==>  36
미래에셋대우, 온라인 고객 제안 접수...개선과제 도출  ==>  43
美 바이오 네오이뮨텍, IPO 추가 주관사 미래에셋대우 선정  ==>  40
미래에셋대우, 다이렉트 신용·담보대출 연 3.9% 이벤트  ==>  89
미래에셋대우, `다이렉트 신용융자·담보·매도담보대출 연 3.9% 이벤...  ==>  100
미래에셋대우, '다이렉트 신용융자·담보·매도담보대출 연3.9%' 이벤...  ==>

유사도는 0부터 100사이 값  
유사도가 50이상이면 걸러지도록 해보자

In [0]:
'''
로직 요약
1. for문으로 뉴스 하나씩 순회(타겟뉴스 만들기)
2. 타겟뉴스와 나머지 뉴스 중 유사도 최대값이 50이상이면 타겟뉴스는 삭제(유사한 뉴스가 있는 것으로 판단)
3. 50미만이라면 클린뉴스에 추가한다(유사한 뉴스가 없는 것으로 판단)
'''
def news_similar(one, alls):
  similar = [ fuzz.token_set_ratio(one, a) for a in alls ]
  return max(similar)

news_clean = [] 
for i, t in enumerate(news_titles):  
  if i == len(news_titles) - 1:                  # 마지막 뉴스는 나머지 전체 뉴스에서 유사도 최대값 50이상이면 클린뉴스 추가
    if news_similar(t, news_titles[:-1]) >= 50:
      news_clean.append(t)
  elif news_similar(t, news_titles[i+1:]) < 50:  # 나머지 뉴스는 유사도 최대값 50미만이면 클린뉴스 추가
    news_clean.append(t)
  
news_clean

['김연추 미래에셋대우 에쿼티본부장 파생결합증권 획일화된 상품 구조 문제...',
 '고광범 미래에셋대우 글로벌 이사 현지화 역량 강화해 직접 딜소싱 늘려...',
 '미래에셋대우, 업계 첫 MTS 해외송금 서비스 개시',
 '미래에셋대우, 온라인 고객 제안 접수...개선과제 도출',
 '美 바이오 네오이뮨텍, IPO 추가 주관사 미래에셋대우 선정',
 "미래에셋대우, '다이렉트 신용융자·담보·매도담보대출 연3.9%' 이벤...",
 '미래에셋대우, 투자성과 가시화 단계 진입-대신',
 "[클릭 e종목]미래에셋대우, 해외IB·네이버·레버리지비율 '재료'충...",
 '미래에셋대우 영통·광화문WM, 22일 투자설명회 개최']

성공! 29개 뉴스를 10개로 줄였다 ㅎㅎㅎㅎ

--- 
### 3. 두번째 과제

표절 코멘트 찾기  

- YES24에서 서평 코멘트 11개를 가져온다.
- 코멘트 중 일부 내용을 조금씩 갖다 붙여 '표절 코멘트'를 만든다.
- fuzzywuzzy로 표절 코멘트를 찾아보자!


* 참고(서평 코멘트 샘플링 도서) : 90년대생이 온다

In [0]:
# YES24 실제 코멘트
comments = \
["★읽기 전에 주의: 편집자의 세심한 편집 없이, 저자 마음대로 쓴 리뷰(!)입니다.근데 저자가 직접 리뷰를 쓰는 경우가 있나?[초등학생 장래희망 1위: 공무원(출처: SBS 세대공감 1억 퀴즈쇼 방송 / 2012년 방영분 캡쳐)]이 책은 (기성세대들이 보기에 한심하기 짝이 없게도) 9급 공무원을 자기들의 꿈이라고 당당하게 이야기하는 1990년생들에 대한 이야기를 담고...",
 "90년생 아이들을 둔 아버지로서 자녀들이 말하는 언어를 이해하고행동하는지 이해 할 수 있는 좋은 책입니다.앞으로 우리 사회와 기업이 추구해야 할 방향을 제시해 줘서 너무 좋습니다. 많은 사람들이 읽었으면 합니다.저에 자녀 모두(3명) 읽도록 했습니다. 자녀들이 이 책을 읽고 어떤 느낌과 생각들이 들었는지 나누기로 하였습니다.이 책을 읽기 전에는 자녀들...",
 "요즘들어 부쩍 어디를 가도 90년대생 친구들과같은 공간 안에 있을 경우가 많이 있더라구요.그런데 90년대생 친구들과 대화가 잘이어지지않을 때가 간혹가다 있죠. 비록 저와 그렇게 큰나이차가 있는건 아니지만 확실히 세대차이가있다는 것을 느끼는 부분이었어요.그래서 90년대생 친구들은 어떠한 마인들를가지며, 생각하는지 궁금했어요. 물론 사람마다차이가 있...",
 "나의 이야기를 다룬 책이라고 생각해서 읽어보니,생각보다 슬프고 짠했다.정말 현실 그대로를 녹여낸 듯한 책이었다.90년대에 태어난 사람들이 왜 이러한 생각을 가지고 있는지,성장 과정과 배경이 어떤 영향을 주었는지, 하나씩 설명하고 있다또 지금 90년대생이 살아가고 있는 환경은 어떠한지도 알려주고 있다.읽고 나니, 내가 왜 이런 생각을 하게 되었는지 알게...",
 '다양한 나이대의 직원들과 일하다보니 하나의 상황을 놓고 세대차이를 느끼곤 한다.얼마 전 야근을 하느라 저녁을 먹고 들어오는데 해가 지는 가운데 비가 내리며 붉은 기운이 도는 것이어쩐지 좀 비현실적인 모습이어서 "귀신나올 것 같다"며 농담을 했는데,한 직원이 갑자기 최근 TV에서 "M"을 봤다며 주제가를 부르는 것이 아닌가.막내 직원에게 심은하를 아냐고..',
 "기성세대의 입장에서 보면 철없는 젊은이들이 마음에 들지 않는 법이다.유사 이래 그래 왔다. 90년대생 자녀가 있는 나로서도 여러 면들이 자식의 행동이 마음에 들지 않는다. 패기도 없고, 꿈도 없고, 결단력도 없어 보인다. 그저 하루하루 편하게 지내면 그만인 모양이다. 남의 간섭을 가급적 받지 않기 위해 9급 공무원이 되는 것이 꿈이라는 젊은이들도 많다. 기...",
 "나는 83년생인데...예전에 79년생 사촌언니가 80년대에도 아이들이 태어났었냐고 말한 적이 있었는데이제 나는 사회에서도 중간 위치이고 90년대생이 사회에 진출한다고 하니...얼마나 세대차이가 있었으면 이러한 책을 쓰셨을까 하고 궁금하기도 하고 90년대생의 특징이 궁금하기도 해서 책을 사서 읽고 있다.나도 계속 근무하다보면 언젠가 우리 팀에도 90년대생이...",
 "이 책에 대해서 워낙 많은 사람들이 말하고 있기 때문에 호기심에 나도 읽게 되었다. 이런 책은 한 번쯤 읽어줘야 해!!라는 단순한 선택을 했지만, 읽으면서 왜 이 책이 많은 사람들이 읽게 되었는지 알 것 같다는 생각이 들었다. 이 책을 재미로 읽어볼까 하는 사람은 읽지 않는 것을 추천한다. 이 책은 재미로 읽기에는 그 분량이 어마어마하고, 또 그다지 재미가...",
 "#90년생이온다 #임홍택지음 #whalebooks #간단함 #병맛 #솔직함 #기업의흥망성쇠 #70년대생X세대 #80년대생밀레니얼세대 #90년대생 #그들을무엇이라부를것인가 #감사 #경제경영서 #융합 #에세이아님 #ㅃㄷ #빡독 #신박사tv #졸꾸정신 #졸꾸러기 #체인지그라운드 #뼈아대 #북스타그램 #책스타그램 #독서는나의힘 #2019년41번째책 #베트남 #하노이 #주재원출장.??????...",
 "90년대생을 앱세대라고 합니다.스마트폰에 익숙한 세대라고 그렇게 부른다고 합니다.공무원 시험에 몰두하고, 대기업에 합격하고도 얼마 못가 그만두기도 합니다.90년대생들은 70년대 이전 세대들이 겪은 1997년 IMF나 90년대생들도 겪은 리먼사태를 통해 오랫동안 일자리가 보장되는공무원에 관심을 가지게 된겁니다.또한 꼰대들이 장악한 여러 기업들에는 오래 일...",
 "왜그럴까 어쩌면 나도 모르게 꼰대가 되어버린건아닐까이해못하는 스스로가 답답했는데 이책을 읽고 나도 이해되고 그들도 이해되고직장에서 많은 도움이됩니다그들에 당돌함이 합리적으로 보이고그들에 용감함이 정당하게 보입니다76년생 동생에게서 90년생이 보이는건비단 나이가아니라 현재를 살아가는사람들에 생각이 반영된게 아닐까 생각해봅니다간혹 내모습에..."]

# 4개 코멘트 일부 내용을 이어 붙여 표절 코멘트 생성
fake = ['편집자의 세심한 편집 없이, 저자 마음대로 쓴 리뷰(!)입니다.나의 이야기를 다룬 책이라고 생각해서 읽어보니,생각보다 슬프고 짠했다.기성세대의 입장에서 보면 철없는 젊은이들이 마음에 들지 않는 법이다.90년대생을 앱세대라고 합니다.왜그럴까 어쩌면 나도 모르게 꼰대가 되어버린건아닐까이해못하는 스스로가 답답했는데']

comments_all = comments + fake

총 12개 코멘트별로 나머지 11개와의 유사도를 뽑아 대푯값을 분석하자

In [0]:
def com_similar(i):
  temp = copy.deepcopy(comments_all)
  target = temp[i]  
  temp.pop(i)    # i만 뺀다  
  similar = [ fuzz.token_set_ratio(target, a) for a in temp ]
  return [min(similar), max(similar), np.mean(similar), np.median(similar)]

results = []
for i in range(len(comments_all)):
  results.append(com_similar(i))

결과물 짜잔~(11번 맨 마지막 코멘트가 표절)

In [0]:
df = pd.DataFrame(results, columns=['min','max', 'mean', 'median'])
df 

,min,max,mean,median
0,11,35,20.272727,18.0
1,11,33,20.818182,20.0
2,12,29,20.090909,20.0
3,8,37,23.090909,23.0
4,5,26,14.363636,12.0
5,13,36,22.090909,21.0
6,17,33,22.818182,21.0
7,6,33,18.545455,18.0
8,7,24,13.909091,12.0
9,11,23,16.363636,15.0


표절 코멘트는 바로 너! 

In [0]:
# 유사도 평균이 가장 큰 코멘트는? 
idxmax = df['mean'].idxmax()
print("인덱스 : ", idxmax)
comments_all[idxmax]

인덱스 :  11


'편집자의 세심한 편집 없이, 저자 마음대로 쓴 리뷰(!)입니다.나의 이야기를 다룬 책이라고 생각해서 읽어보니,생각보다 슬프고 짠했다.기성세대의 입장에서 보면 철없는 젊은이들이 마음에 들지 않는 법이다.90년대생을 앱세대라고 합니다.왜그럴까 어쩌면 나도 모르게 꼰대가 되어버린건아닐까이해못하는 스스로가 답답했는데'